# AOC2022

## Day 7 / Part 1 / No Space Left On Device

Problem Description: https://adventofcode.com/2022/day/7

Input: [Example](aoc2022_day7_example.txt)

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
"""Solution for AOC2022, day 7, part 1."""
from abc import ABC
import logging
import os
import re
import sys

LOGGER = logging.getLogger(__name__)

# show/hide debug logs
SHOW_DEBUG_LOG = False
# set input file
INPUT_FILE = "aoc2022_day7_example.txt"

In [3]:
DIR_MAX_SIZE = 100000

In [4]:
class Node(ABC):
    """Generic node of the filesystem."""
    node_type = None
    name = None
    size = 0
    parent = None
    children = []

    def update_size(self):
        """Update the size of the node."""
        if self.node_type == "dir":
            self.size = 0
            for child in self.children:
                self.size += child.update_size()
        return self.size

    def find(self, max_size):
        """Recursively find nodes with a total size <= max_size."""
        if self.node_type == "dir" and self.size <= max_size:
            yield self.size
        for child in self.children:
            yield from child.find(max_size)

    def __str__(self, depth=0):
        """Compute the informal string representation of the node."""
        obj_str = ""
        node_details = f"- {self.name} ({self.node_type}, size={self.size})"
        obj_str = "  " * depth + node_details
        for child in self.children:
            obj_str += "\n"
            obj_str += child.__str__(depth+1)
        return obj_str


class Folder(Node):
    """Folder of the filesystem."""
    def __init__(self, name, parent=None, children=None):
        self.node_type = "dir"
        self.name = name
        self.parent = parent
        self.children = children or []


class File(Node):
    """File of the filesystem."""
    def __init__(self, name, size, parent=None):
        self.node_type = "file"
        self.name = name
        self.size = size
        self.parent = parent
        self.children = []

In [5]:
def change_dir(path, directory):
    """Changes current directory."""
    if directory == "/":
        return "/"
    if directory == "..":
        return os.path.normpath(os.path.join(path, "..")).replace("\\", "/")
    return os.path.normpath(os.path.join(path, directory)).replace("\\", "/")

In [6]:
def main():
    """Main function to solve puzzle."""
    curr_path = None
    node_map = {"/": Folder("/")}
    LOGGER.debug("traverse filesystem...")
    with open(INPUT_FILE, encoding="utf-8") as file_obj:
        line = file_obj.readline().rstrip()
        while line:
            if line.startswith("$ cd"):
                curr_path = change_dir(curr_path, line[5:])
                LOGGER.debug("  %s", curr_path)
                if curr_path not in node_map:
                    parent_path = os.path.dirname(curr_path)
                    dir_name = os.path.basename(curr_path)
                    parent_node = node_map[parent_path]
                    curr_node = Folder(dir_name, parent_node)
                    node_map[curr_path] = curr_node
                    parent_node.children.append(curr_node)
                line = file_obj.readline().rstrip()

            if line.startswith("$ ls"):
                line = file_obj.readline().rstrip()
                while line and not line.startswith("$"):
                    if line.startswith("dir"):
                        pass
                    else:
                        file_size, file_name = re.search(
                            r"(\d+) (.+)", line
                        ).groups()
                        parent_node = node_map[curr_path]
                        file_node = File(
                            file_name, int(file_size), parent_node
                        )
                        parent_node.children.append(file_node)
                    line = file_obj.readline().rstrip()

    node_map["/"].update_size()

    LOGGER.debug(
        "\n"
        "filesystem tree:\n"
        "%s\n",
        node_map["/"]
    )

    small_nodes = node_map["/"].find(DIR_MAX_SIZE)
    print(f"solution: {sum(small_nodes)}")

In [7]:
if __name__ == "__main__":
    LOGGER.setLevel(logging.DEBUG if SHOW_DEBUG_LOG else logging.INFO)
    log_formatter = logging.Formatter("%(message)s")
    log_handler = logging.StreamHandler(sys.stdout)
    log_handler.setFormatter(log_formatter)
    LOGGER.addHandler(log_handler)
    main()

solution: 95437
